In [ ]:
!kaggle datasets download ujjwalchowdhury/yoga-pose-classification

Dataset URL: https://www.kaggle.com/datasets/ujjwalchowdhury/yoga-pose-classification
License(s): CC0-1.0
 94% 14.0M/15.0M [00:00<00:00, 70.8MB/s]
100% 15.0M/15.0M [00:00<00:00, 69.6MB/s]


In [ ]:
!unzip /content/yoga-pose-classification.zip

Archive:  /content/yoga-pose-classification.zip
  inflating: YogaPoses/Downdog/00000000.jpg  
  inflating: YogaPoses/Downdog/00000001.jpg  
  inflating: YogaPoses/Downdog/00000002.jpg  
  inflating: YogaPoses/Downdog/00000003.jpg  
  inflating: YogaPoses/Downdog/00000004.jpg  
  inflating: YogaPoses/Downdog/00000007.jpg  
  inflating: YogaPoses/Downdog/00000008.jpg  
  inflating: YogaPoses/Downdog/00000009.jpg  
  inflating: YogaPoses/Downdog/00000010.jpg  
  inflating: YogaPoses/Downdog/00000011.jpg  
  inflating: YogaPoses/Downdog/00000012.jpg  
  inflating: YogaPoses/Downdog/00000014.jpg  
  inflating: YogaPoses/Downdog/00000016.jpg  
  inflating: YogaPoses/Downdog/00000017.jpg  
  inflating: YogaPoses/Downdog/00000018.jpg  
  inflating: YogaPoses/Downdog/00000019.jpg  
  inflating: YogaPoses/Downdog/00000020.jpg  
  inflating: YogaPoses/Downdog/00000022.jpg  
  inflating: YogaPoses/Downdog/00000023.jpg  
  inflating: YogaPoses/Downdog/00000024.jpg  
  inflating: YogaPoses/Downdog/0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset="/content/YogaPoses"

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.metrics import Accuracy
import numpy as np

In [ ]:
xcep = Xception(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for layer in xcep.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xcep.output)

In [ ]:
prediction = Dense(5, activation='softmax')(x)

In [ ]:
model = Model(inputs=xcep.input, outputs=prediction)

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1 (Conv2D)     │ (None, 111, 111, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_bn           │ (None, 111, 111, 32)   │            128 │ block1_conv1[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv1_act          │ (None, 111, 111, 32)   │              0 │ block1_conv1_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2 (Conv2D)     │ (None, 109, 109, 64)   │         18,432 │ block1_conv1_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_bn           │ (None, 109, 109, 64)   │            256 │ block1_conv2[0][0]     │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1_conv2_act          │ (None, 109, 109, 64)   │              0 │ block1_conv2_bn[0][0]  │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1           │ (None, 109, 109, 128)  │          8,768 │ block1_conv2_act[0][0] │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv1_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv1[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_act       │ (None, 109, 109, 128)  │              0 │ block2_sepconv1_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2           │ (None, 109, 109, 128)  │         17,536 │ block2_sepconv2_act[0… │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_sepconv2_bn        │ (None, 109, 109, 128)  │            512 │ block2_sepconv2[0][0]  │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 55, 55, 128)    │          8,192 │ block1_conv2_act[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block2_pool               │ (None, 55, 55, 128)    │              0 │ block2_sepconv2_bn[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization  

 Total params: 21,363,245 (81.49 MB)

 Trainable params: 501,765 (1.91 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data = ImageDataGenerator(horizontal_flip=True, shear_range=0.2,zoom_range=0.2,rescale=1./255, validation_split=0.3)

In [ ]:
training_set = data.flow_from_directory(dataset, target_size=(224, 224), batch_size=16, class_mode='categorical', subset='training')

Found 694 images belonging to 5 classes.


In [ ]:
testing_set = data.flow_from_directory(dataset, target_size=(224, 224), batch_size=16, class_mode='categorical', subset='validation')

Found 294 images belonging to 5 classes.


In [ ]:
len(training_set), len(testing_set)

(44, 19)

In [ ]:
model.fit(training_set, validation_data=testing_set, epochs=26, steps_per_epoch=len(training_set)//16, validation_steps=len(testing_set)//16)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 21s 11s/step - accuracy: 0.1458 - loss: 3.9184 - val_accuracy: 0.2500 - val_loss: 8.8250
Epoch 2/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 13s 9s/step - accuracy: 0.3750 - loss: 9.2875 - val_accuracy: 0.3750 - val_loss: 7.2616
Epoch 3/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 19s 15s/step - accuracy: 0.4792 - loss: 4.2595 - val_accuracy: 0.5625 - val_loss: 2.5970
Epoch 4/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 7s/step - accuracy: 0.4792 - loss: 3.3624 - val_accuracy: 0.6875 - val_loss: 1.6334
Epoch 5/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 9s/step - accuracy: 0.7500 - loss: 1.0361 - val_accuracy: 0.6250 - val_loss: 3.3915
Epoch 6/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 7s/step - accuracy: 0.8333 - loss: 1.9694 - val_accuracy: 0.6250 - val_loss: 2.6711
Epoch 7/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 20s 15s/step - accuracy: 0.7083 - loss: 1.8890 - val_accuracy: 0.8125 - val_loss: 0.4051
Epoch 8/26
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 9s/step - accuracy: 0.7083 - loss: 1.9884 - val_accuracy: 1.0000 - val_loss: 0.0609
Epoch

In [ ]:
model.save('xcep_yoga.h5')

In [ ]:
model = load_model('xcep_yoga.h5')

In [ ]:
img = load_img('/content/YogaPoses/Downdog/00000000.jpg', target_size=(224, 224))

In [ ]:
img = image.img_to_array(img)

In [ ]:
img.shape

(224, 224, 3)

In [ ]:
x = np.expand_dims(img, axis=0)
img_data = preprocess_input(x)
img_data.shape

(1, 224, 224, 3)

In [ ]:
pred = model.predict(img_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [58]:
p= np.argmax(pred)

In [59]:
columns=['Downdog','Goddess','Plank','Tree','Warrior2']

In [60]:
result = str(columns[p])

In [ ]:
result

'Downdog'